In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\MLOPS\\End-To-End-MlOps-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [19]:
@dataclass
class ModelTrainConfig:
   dir:Path
   train_arr: Path
   test_arr: Path
   model:  Path
   n_estimators: int
   max_depth: int
   min_samples_split:int
   min_samples_leaf:int

In [7]:
from src.ds.utils.utils import create_dir,read_yaml
from src.ds.constants.yaml_path import *
from src.ds import logging

In [30]:
class ConfigManager:
    def __init__(self,config_file_path=Config_file_path,prams_file_path=Param_file_path,schema_file_path=Schema_file_path) -> None:
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(prams_file_path)
        self.schema=read_yaml(schema_file_path)

        create_dir([self.config.artifacts_root])
    def get_model_trainer_config(self):
      try:
         config=self.config.Model_Trainer
         params=self.params.RandomForestClassifier
         create_dir([config.dir])
         model_train_config=ModelTrainConfig(
               dir=config.dir,
               train_arr=config.train_arr,
               test_arr=config.test_arr,
               model=config.model,
               n_estimators=params.n_estimators,
               max_depth=params.max_depth,
               min_samples_leaf=params.min_samples_leaf,
               min_samples_split=params.min_samples_split
			)
         return model_train_config
      except Exception as e:
         logging.info(f'error in MODEL TRAIN config {str(e)}')
         raise e

In [38]:
import joblib
import numpy as np
from src.ds.utils.utils import model_evaluatuion,save_obj
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report

In [39]:
class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def initiate_model_training(self):
        try:
            # Load data
            train_array = np.load(self.config.train_arr,allow_pickle=True)
            test_array = np.load(self.config.test_arr,allow_pickle=True)
            logging.info('Array data loaded')

            # Split data into features and target
            x_train, y_train = train_array[:, :-1], train_array[:, -1]
            x_test, y_test = test_array[:, :-1], test_array[:, -1]

            # Initialize and train model
            model = RandomForestClassifier(
                n_estimators=self.config.n_estimators,
                min_samples_leaf=self.config.min_samples_leaf,
                max_depth=self.config.max_depth,
                min_samples_split=self.config.min_samples_split
            )
            model.fit(x_train, y_train)

            # Predict and calculate accuracy
            y_pred = model.predict(x_test)
            accuracy = accuracy_score(y_test, y_pred)
            print(f'{model} model accuracy: {accuracy}')

            # Save the model
            save_obj(
                file_path=self.config.model,
                obj=model
				)
            logging.info('Model saved successfully')

        except Exception as e:
            logging.info(f'Error in model training: {str(e)}')
            raise e

In [41]:
try:
		config=ConfigManager()
		model_trainer_config=config.get_model_trainer_config()
		model_trainer=ModelTrainer(config=model_trainer_config)
		model_trainer.initiate_model_training()
except Exception as e:
         logging.info('error')

[2024-11-06 17:43:46,412: INFO :utils : Yaml file config\config.yaml created]
[2024-11-06 17:43:46,422: INFO :utils : Yaml file params.yaml created]
[2024-11-06 17:43:46,430: INFO :utils : Yaml file schema.yaml created]
[2024-11-06 17:43:46,432: INFO :utils : created directory at: artifacts]
[2024-11-06 17:43:46,435: INFO :utils : created directory at: artifacts/model_Trainer]
[2024-11-06 17:43:46,442: INFO :3824806231 : Array data loaded]
RandomForestClassifier(max_depth=10, min_samples_leaf=3, min_samples_split=5,
                       n_estimators=300) model accuracy: 0.81875
[2024-11-06 17:43:47,089: INFO :3824806231 : Model saved successfully]
